# CTF分享--ANGR模块的利用
## angr是什么？
angr是一个支持多架构的二进制代码分析工具，能够在二进制文件上进行动态符号执行(如Mayhem、KLEE等)和各种静态分析。
## angr分析的基本流程
1. 将二进制程序载入angr分析系统
2. 将二进制程序转换成中间语言（intermediate representation, IR）
3. 将IR语言转换成语义较强的表达形式，比如，这个程序做了什么，而不是它是什么。
4. 执行进一步的分析，比如，完整的或者部分的静态分析（依赖关系分析，程序分块）、程序空间的符号执行探索（挖掘溢出漏洞）、一些对于上面方式的结合。

# angr主要模块介绍
## Project
angr.Project可以用来加载一个二进制文件，同时，proj自身也有一些属性

In [125]:
import angr
import networkx as nx
from angrutils import *

In [126]:
# proj = angr.Project("../problems/00_angr_find",auto_load_libs=False)
proj = angr.Project("../problems/00_angr_find")
print(proj.arch) 
print(hex(proj.entry)) 
print(proj.filename) 

<Arch X86 (LE)>
0x8048450
../problems/00_angr_find


## loader
angr中有名为CLE的模块，代表“CLE Loads Everything”，负责获取二进制文件（以及它所依赖的任何库）并以易于使用的方式将其呈现给angr的其余部分。\
angr使用CLE模块来处理从二进制文件加载到在虚拟地址空间的问题，而CLE的结果会被呈现在.loader属性中。 \
用它来查看angr加载程序时同时加载的共享库，并对加载的地址空间进行基本的查询。

In [127]:
print(proj.loader)
print(proj.loader.shared_objects) # may look a little different for you!
print(hex(proj.loader.min_addr))
print(hex(proj.loader.max_addr))
print(proj.loader.main_object) # we've loaded several binaries into this project. Here's the main one!
print(proj.loader.main_object.execstack) # sample query: does this binary have an executable stack?
print(proj.loader.main_object.pic) # sample query: is this binary position-independent(位置无关)?

<Loaded 00_angr_find, maps [0x8048000:0x8608000]>
OrderedDict([('00_angr_find', <ELF Object 00_angr_find, maps [0x8048000:0x804a03f]>), ('libc.so.6', <ELF Object libc-2.30.so, maps [0x8100000:0x82e3f4b]>), ('ld-linux.so.2', <ELF Object ld-2.30.so, maps [0x8300000:0x832993f]>), ('extern-address space', <ExternObject Object cle##externs, maps [0x8400000:0x8408000]>), ('cle##tls', <ELFTLSObjectV2 Object cle##tls, maps [0x8500000:0x8514808]>)])
0x8048000
0x8608000
<ELF Object 00_angr_find, maps [0x8048000:0x804a03f]>
False
False


## block类
block是angr进行分析的最小单位，factory.block(addr)将从给定的地址提取出该地址的代码块。通过生成的block类，可以获取关于代码块的信息

In [128]:
block = proj.factory.block(proj.entry)
print(block.pp()) #disasm
print(block.bytes) # b'1\xed^\x89\xe1\x83\xe4\xf0PTRh ...'
print(block.instructions) #number of instructions # 13
print(block.instruction_addrs) # addresses of the instructions

0x8048450:	xor	ebp, ebp
0x8048452:	pop	esi
0x8048453:	mov	ecx, esp
0x8048455:	and	esp, 0xfffffff0
0x8048458:	push	eax
0x8048459:	push	esp
0x804845a:	push	edx
0x804845b:	push	0x8048710
0x8048460:	push	0x80486b0
0x8048465:	push	ecx
0x8048466:	push	esi
0x8048467:	push	0x80485c7
0x804846c:	call	0x8048420
None
b'1\xed^\x89\xe1\x83\xe4\xf0PTRh\x10\x87\x04\x08h\xb0\x86\x04\x08QVh\xc7\x85\x04\x08\xe8\xaf\xff\xff\xff'
13
[134513744, 134513746, 134513747, 134513749, 134513752, 134513753, 134513754, 134513755, 134513760, 134513765, 134513766, 134513767, 134513772]


## States
当一个项目被传入angr.Project()后，其代表的只是程序的静态镜像，要想让程序成功执行，还需要另外的数据结构来保存执行时程序的各个状态，这就是States类的作用
在States中保存了程序运行时的寄存器和内存状态，其中,BV代表的是一个bitvector。

In [129]:
state = proj.factory.entry_state()
print(state.regs.ip) # get the current instruction pointer
print(state.regs.ax)
print(state.mem[proj.entry].int.resolved)

<BV32 0x8048450>
<BV16 0x1c>
<BV32 0x895eed31>


## Bitvector
可以指定位数的二进制向量

In [114]:
bv = state.solver.BVV(0x1234, 32)
print(bv)
# create a 32-bit-wide bitvector with value 0x1234
# BVV stands for bitvector value
print(hex(state.solver.eval(bv)))
# convert to python int
state.regs.si = state.solver.BVV(3, 32)
state.mem[0x1000].long = 4
print(state.mem[0x1000].long)
print(state.mem[0x1000].long.resolved)

<BV32 0x1234>
0x1234
<long (32 bits) <BV32 0x4> at 0x1000>
<BV32 0x4>


## Simulation Managers
simulation manager(simgr)主要为States的执行提供接口，用step()可以单步执行 \
之前提到过，angr可用于动态符号执行，而simgr.active是保存simgr中目前存在状态(states)的数组,在遇到诸如分支语句时，会创建一个新的状态。\
每个状态有专门的Stash types,来表示该状态能否被继续执行(etc. active,deadended)

In [123]:
state = proj.factory.entry_state()
simgr = proj.factory.simgr(state)
print(simgr.active[0].regs.ip)
simgr.step()
print(simgr.active[0].regs.ip)
print(simgr)
while len(simgr.active) == 1:
    simgr.step()
print(simgr)
simgr.run()
print("after run()")
print(simgr)

<BV32 0x8048450>
<BV32 0x8048420>
<SimulationManager with 1 active>


WARNING | 2020-08-09 15:55:23,937 | angr.state_plugins.symbolic_memory | Filling register edi with 4 unconstrained bytes referenced from 0x80486b1 (__libc_csu_init+0x1 in 00_angr_find (0x80486b1))
WARNING | 2020-08-09 15:55:23,945 | angr.state_plugins.symbolic_memory | Filling register ebx with 4 unconstrained bytes referenced from 0x80486b3 (__libc_csu_init+0x3 in 00_angr_find (0x80486b3))


<SimulationManager with 2 active>


WARNING | 2020-08-09 15:55:28,364 | angr.state_plugins.symbolic_memory | Filling memory at 0x7ffefffc with 71 unconstrained bytes referenced from 0x8187ce0 (strcmp+0x0 in libc.so.6 (0x87ce0))
WARNING | 2020-08-09 15:55:28,366 | angr.state_plugins.symbolic_memory | Filling memory at 0x7ffeff50 with 4 unconstrained bytes referenced from 0x8187ce0 (strcmp+0x0 in libc.so.6 (0x87ce0))


after run()
<SimulationManager with 18 deadended>


## Analyses
proj.analyses提供了许多自带的分析模块,这里以绘制控制流图(CFG)为例：

In [130]:
import angr
from angrutils import *
main = proj.loader.main_object.get_symbol("main")
start_state = proj.factory.blank_state(addr=main.rebased_addr)
cfg = proj.analyses.CFGEmulated(fail_fast=True, starts=[main.rebased_addr], initial_state=start_state)
plot_cfg(cfg, "example", asminst=True, remove_imports=True, remove_path_terminator=True)  

# 案例
## 案例一：find的应用(00_angr_find)
## 案例二：通过输出判断程序正确性(02_angr_find_condition) 
## 案例三：手动构造寄存器中的变量(03_angr_symbolic_registers) 
## 案例四：手动构造堆栈中的变量(04_angr_stack) 
## 案例五：手动构造内存中的变量(05_angr_symbolic_memory) 

# 附录
- Angr：一个具有动态符号执行和静态分析的二进制分析工具
https://www.freebuf.com/sectool/143056.html
- angr文档
https://docs.angr.io/core-concepts/toplevel
- API文档
http://angr.io/api-doc/angr.html#module-angr.project